# Agent Notebook
This notebook demonstrates an agent workflow using autogen and models hosted on Ollama.

In [48]:
!pip install -U autogen ag2[openai] autogen-ext[ollama] jupytext markdownify accelerate bitsandbytes


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports and constants

In [49]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
# local fallback client if library missing or openai unavailable
try:
    from autogen import OpenAIChatCompletionClient as _OAIClient
    import openai
    OpenAIChatCompletionClient = _OAIClient
except Exception:
    class OpenAIChatCompletionClient:
        def __init__(self, model, base_url=None, api_key=None):
            self.model = model
            self.base_url = base_url
            self.api_key = api_key
        def chat(self, messages, temperature=0):
            raise NotImplementedError('OpenAIChatCompletionClient is unavailable')
import pathlib, datetime, uuid, subprocess, shlex, markdownify

REPO_ROOT = pathlib.Path.cwd()
LOG_DIR = REPO_ROOT / 'agent_logs'
LOG_DIR.mkdir(exist_ok=True, parents=True)
BASE_URL = 'http://docker-ai:11434/v1'


## Model clients

In [50]:
qwen14_client = OpenAIChatCompletionClient(
    model='Qwen2.5-14B-Instruct-1M-Q8_0:latest',
    base_url=BASE_URL,
    api_key='ollama')

qwen32_client = OpenAIChatCompletionClient(
    model='Qwen3-32B-Q5_0:latest',
    base_url=BASE_URL,
    api_key='ollama')

devstral_client = OpenAIChatCompletionClient(
    model='devstral:24b',
    base_url=BASE_URL,
    api_key='ollama')

## Markdown logger

In [51]:
def log_markdown(task_id, role, content):
    ts = datetime.datetime.utcnow().isoformat()
    fn = LOG_DIR / f"{task_id}.md"
    if not fn.exists():
        with open(fn, 'w') as f:
            f.write(f'---\nid: {task_id}\ncreated: {ts}\n---\n\n')
    with open(fn, 'a') as f:
        f.write(f'### {ts} — {role}\n\n{markdownify.markdownify(content)}\n\n')
    print(f'[{ts}] {role}: {content}')


## Shell helper

In [52]:
def run_shell(cmd: str) -> str:
    out = subprocess.check_output(shlex.split(cmd), text=True, timeout=900, stderr=subprocess.STDOUT)
    return f'```shell\n$ {cmd}\n{out}\n```'


## Agent declarations

In [53]:
planner = AssistantAgent(
    name='planner',
    llm_config={'config_list': [{ 'model': qwen14_client.model, 'base_url': qwen14_client.base_url, 'api_key': qwen14_client.api_key }], 'temperature': 0.3},
    system_message=("You are a project planner. Break the user's request into a YAML list of atomic tasks. Stop when each sub-task can be executed in one short Python call or shell command inside the current Jupyter kernel."),
)

worker = AssistantAgent(
    name='worker',
    llm_config={'config_list': [{ 'model': qwen32_client.model, 'base_url': qwen32_client.base_url, 'api_key': qwen32_client.api_key }], 'temperature': 0},
    system_message='Execute the given atomic task and return result.')
worker.register_function({'run_shell': run_shell})

coder = AssistantAgent(
    name='coder',
    llm_config={'config_list': [{ 'model': devstral_client.model, 'base_url': devstral_client.base_url, 'api_key': devstral_client.api_key }], 'temperature': 0},
    system_message='You are a senior software engineer. Write, refactor, and debug code snippets as requested.')

reviewer = AssistantAgent(
    name='reviewer',
    llm_config={'config_list': [{ 'model': qwen14_client.model, 'base_url': qwen14_client.base_url, 'api_key': qwen14_client.api_key }], 'temperature': 0},
    system_message=("Evaluate the worker or coder output against the task description. If incorrect, respond with REVISE and instructions; otherwise APPROVED."),
)

agents = [planner, worker, coder, reviewer]
group = GroupChat(agents=agents, max_round=30)
manager = GroupChatManager(groupchat=group, llm_config={'config_list': [{ 'model': qwen14_client.model, 'base_url': qwen14_client.base_url, 'api_key': qwen14_client.api_key }], 'temperature': 0})
proxy = UserProxyAgent(name='user', human_input_mode='NEVER', code_execution_config={'use_docker': False})


## Driver function

In [54]:
def run_agent(prompt: str):
    task_id = uuid.uuid4().hex[:8]
    log_markdown(task_id, 'INFO', 'Agent run started')
    log_markdown(task_id, 'USER', prompt)
    proxy.initiate_chat(manager, message=prompt)
    for m in group.chat_history:
        log_markdown(task_id, m['role'], m['content'])
    log_markdown(task_id, 'INFO', 'Agent run complete')
    return LOG_DIR / f'{task_id}.md'


## Example call

In [55]:
run_agent('Generate Python code to scrape example.com daily and store results in SQLite …')

[2025-07-07T03:03:12.216463] INFO: Agent run started
[2025-07-07T03:03:12.217463] USER: Generate Python code to scrape example.com daily and store results in SQLite …
user (to chat_manager):

Generate Python code to scrape example.com daily and store results in SQLite …

--------------------------------------------------------------------------------


C:\Users\CJ\AppData\Local\Temp\ipykernel_40000\1866636929.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow().isoformat()


[autogen.oai.client: 07-06 20:04:39] {708} WARNING - Model Qwen2.5-14B-Instruct-1M-Q8_0:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 07-06 20:04:39] {708} WARNING - Model Qwen2.5-14B-Instruct-1M-Q8_0:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 07-06 20:04:40] {708} WARNING - Model Qwen2.5-14B-Instruct-1M-Q8_0:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: planner

[autogen.oai.client: 07-06 20:05:43] {708} WARNING - Model Qwen2.5-14B-Instruct-1M-Q8_0:latest is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for 

## Git auto-commit

In [ ]:
!git add agent_logs/*.md && (git diff --cached --quiet || git commit -m 'agent run') && git push

## Version info

In [ ]:
!pip freeze | grep -E '(autogen|transformers|ollama)'